### Setup

In [71]:
# Reload the dependencies
%reload_ext autoreload
%autoreload 2

import sys
from typing import Dict

sys.path.insert(0, "../..")
from src.utils import MilvusUtil

# Options
embedding_model = MilvusUtil.text_embedding_3_large
dim = MilvusUtil.get_dimensions_of_model(embedding_model)


### Color function

In [72]:

# Use a scale of 0 to 1 to create a text color code from red to yellow to green, continuously
def get_color_code(value: float) -> str:
    if value < 0 or value > 1:
        raise ValueError("Value must be between 0 and 1")
    ranges = [0, 0.25, 0.5, 1]
    if value < ranges[1]:
        # Red (255,0,0) to Orange (255,165,0)
        ratio = value / ranges[1]
        r = 255
        g = int(165 * ratio)
        b = 0
    elif value < ranges[2]:
        # Orange (255,165,0) to Yellow (255,255,0)
        ratio = (value - ranges[1]) / (ranges[2] - ranges[1])
        r = 255
        g = int(165 + (90 * ratio))
        b = 0
    else:
        # Yellow (255,255,0) to Green (0,255,0)
        ratio = (value - ranges[2]) / (ranges[3] - ranges[2])
        r = int(255 * (1 - ratio))
        g = 255
        b = 0
    return f"\033[38;2;{r};{g};{b}m"

def get_colored_text(text: str, value: float) -> str:
    """
    Print text with a color code based on the value.
    """
    color_code = get_color_code(value)
    return(f"{color_code}{text}\033[0m")

print(get_colored_text("Hello world!", 0.75))

Hello world!


In [ ]:
import time

convo_blurb = "yoga"

brain_blurb = [
    "fitness",
    "women",
    "exercise",
    "meditation",
    "health",
    "spirituality",
    "place to meet people",
    "place to meet women"
]

embedding_main = MilvusUtil.get_embedding(convo_blurb, embedding_model, dim)
embedding_comparisons: Dict[str, float] = {}
print(f"Mentioned in conversation: {convo_blurb}")

class ColNames:
    brain_match = "brain match"
    similarity = "similarity"
    embed_time = "embed time"
    cosine_time = "cosine time"

table_dimensions = {
    ColNames.brain_match: 30,
    ColNames.similarity: len(ColNames.similarity),
    ColNames.embed_time: len(ColNames.embed_time),
    ColNames.cosine_time: len(ColNames.cosine_time)
}

# Print the results in a formatted table (starting with header)
print("+" + "-" * (table_dimensions[ColNames.brain_match] + 2) + "+" +
      "-" * (table_dimensions[ColNames.similarity] + 2) + "+" +
      "-" * (table_dimensions[ColNames.embed_time] + 2) + "+" +
      "-" * (table_dimensions[ColNames.cosine_time] + 2) + "+")
print(f"| {ColNames.brain_match}{' ' * (table_dimensions[ColNames.brain_match] - len(ColNames.brain_match))} | " +
        f"{ColNames.similarity}{' ' * (table_dimensions[ColNames.similarity] - len(ColNames.similarity))} | " +
        f"{ColNames.embed_time}{' ' * (table_dimensions[ColNames.embed_time] - len(ColNames.embed_time))} | " +
        f"{ColNames.cosine_time}{' ' * (table_dimensions[ColNames.cosine_time] - len(ColNames.cosine_time))} |")
print("+" + "-" * (table_dimensions[ColNames.brain_match] + 2) + "+" +
      "-" * (table_dimensions[ColNames.similarity] + 2) + "+" +
      "-" * (table_dimensions[ColNames.embed_time] + 2) + "+" +
      "-" * (table_dimensions[ColNames.cosine_time] + 2) + "+")
for blurb in brain_blurb:
    # Get the embedding for the brain blurb
    embed_time_start = time.time()
    embedding = MilvusUtil.get_embedding(blurb, embedding_model, dim)
    embed_elapsed_time = time.time() - embed_time_start
    embedding_comparisons[blurb] = embedding

    # Calculate the cosine similarity
    cosine_time_start = time.time()
    similarity = MilvusUtil.cosine_similarity(embedding_main, embedding)
    cosine_elapsed_time = time.time() - cosine_time_start

    colored_similarity_value: str = get_colored_text(f"{similarity:.4f}", similarity)

    print(f"| {blurb + ' ' * (table_dimensions[ColNames.brain_match] - len(blurb))} | "
          f"{colored_similarity_value + ' ' * (table_dimensions[ColNames.similarity] + 1 - len(f'{similarity:.4f}s'))} | "
          f"{f'{embed_elapsed_time:.4f}s' + ' ' * (table_dimensions[ColNames.embed_time] - len(f'{embed_elapsed_time:.4f}s'))} | "
          f"{f'{cosine_elapsed_time:.4f}s' + ' ' * (table_dimensions[ColNames.cosine_time] - len(f'{cosine_elapsed_time:.4f}s'))} |")
print("+" + "-" * (table_dimensions[ColNames.brain_match] + 2) + "+" +
      "-" * (table_dimensions[ColNames.similarity] + 2) + "+" +
      "-" * (table_dimensions[ColNames.embed_time] + 2) + "+" +
      "-" * (table_dimensions[ColNames.cosine_time] + 2) + "+")

Mentioned in conversation: yoga
+--------------------------------+------------+------------+-------------+
| brain match                    | similarity | embed time | cosine time |
+--------------------------------+------------+------------+-------------+
| fitness                        | 0.4971     | 0.2979s    | 0.0002s     |
| women                          | 0.2477     | 0.3186s    | 0.0004s     |
| exercise                       | 0.4761     | 0.1964s    | 0.0004s     |
| meditation                     | 0.6542     | 0.4921s    | 0.0004s     |
| health                         | 0.3515     | 0.1652s    | 0.0004s     |
| spirituality                   | 0.5036     | 0.2708s    | 0.0004s     |
| place to meet people           | 0.1907     | 0.2826s    | 0.0003s     |
| place to meet women            | 0.2101     | 0.5063s    | 0.0003s     |
+--------------------------------+------------+------------+-------------+
